# Trigram model

## Task 1: Third-order letter approximation model

In [15]:
import random
import json

First cell defines needed parameters such as paths to files.
A dictionary to add each text to.
And Strings to define preamble and postamble markers for removal.
Parameters for word generation.

In [16]:
#file paths to the texts to be used for the trigram
files = [
    '/workspaces/emergingtechnologies/data/frankenstein.txt',
    '/workspaces/emergingtechnologies/data/bookplates.txt',
    '/workspaces/emergingtechnologies/data/spyglassmoutain.txt',
    '/workspaces/emergingtechnologies/data/theoldhouse.txt',
    '/workspaces/emergingtechnologies/data/windofdestiny.txt',
]
#file path to the list of words to be used for the trigram comparisons
Comparisons = '/workspaces/emergingtechnologies/data/words.txt'

#dictionary to store the texts
texts = {}

#preamble and postamble markers
start_marker = " ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

#parameters for the word generator
startseed = "TH"
length = 10000

#file path to the json file to store the trigram
jsonFile = '/workspaces/emergingtechnologies/trigrams.json'

We then need to read each file. Rather than doing this individually, I've created a loop which adds each text to a dictionary.I've done it this way so I don't have to deal with individual texts and can simply loop over the dictionary when required to retrieve each text.

After this, I used the `.find` feature within the same loop, to locate the markers that I've predefined and extract all text between these markers using `.strip`. This removes the preamble and postamble. I then add this cleaned text to the dictionary with the filename as the key. This allows me to print each text  separately to check each is cleaned and for future use.

In [17]:
#read each file and remove preamble and postamble
for file_path in files:
    with open(file_path, 'r') as f:
        text = f.read()

        #find the start and end markers
        start = text.find(start_marker)
        end = text.find(end_marker)

        #skip the start marker
        start += len(start_marker)
        #extract all text between the markers
        text = text[start:end].strip()

        #store the cleaned text in the dictionary with filename as the key
        filename = file_path.split('/')[-1]  # Get filename from path
        texts[filename] = text

I then create a new loop that iterates over each key-value pair in the `texts` dictionary. I had to add a `.replace` method to this to remove `\n` as it was showing up a number of times. I then define the characters allowed in text to be fed to the model. After converting my text to uppercase, I clean it by removing any characters not in the defined character set. Finally I add the cleaned text back to its corresponding key in the dictionary.

In [18]:
for filename, text in texts.items():
    #remove all newlines
    text = text.replace('\n', ' ')    
    text = text.upper()
    chars = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ. ')
    #remove all characters that are not in the set of allowed characters
    cleaned = ''.join(c for c in text if c in chars)
    #store the cleaned text back in the dictionary
    texts[filename] = cleaned

#print(texts['theoldhouse.txt'])

I then created a trigram model. Using the `model` dictionary and a similar key-value loop as before, I created a loop that iterates over each files text. Each trigram is extracted from the text. A `if` statement was required to remove empty values missed during the cleaning process. These trigrams are then added to the dictionary as a keys, with each occurance being counted and added to the value.

In [19]:
model = {}

for filename, cleaned in texts.items():
  #iterate over the text and count the occurrences of each trigram
  for i in range(len(cleaned) - 2):
    #extract the trigram
    trigram = cleaned[i:i+3]
    if trigram != '' and trigram != '   ':
        #store the trigram in the model
      model[trigram] = model.get(trigram, 0) + 1

This is a sorted example of the model.

In [20]:
#sorted(model.items(), key=lambda x: x[1], reverse=True)

## Task 2: Third-order letter approximation generation

I first created a trigram dictionary to store the bigrams with counts of next characters, I then looped over the existing trigram and extracted each bigram. Next, I collected the character following each bigram and stored it as `next_char`. Finally, I added the bigram as the key, with the next character as a nested key with its count as the value.

In [21]:
#trigram look up dictionary for model
trigram_dict = {}

#loop through the model and its count in the model
for trigram, count in model.items():
    #exsure trigram has 3 characters
    if len(trigram) == 3:
        #extract the bigram
        bigram = trigram[:2]
        #extract the next character
        next_char = trigram[2]
        #check bigram is not already accounted for
        if bigram not in trigram_dict:
            #add the bigram to the dictionary
            trigram_dict[bigram] = {}
        #add the next character to the dictionary
        trigram_dict[bigram][next_char] = count

# Manually format and print the trigram dictionary
#for bigram, next_chars in trigram_dict.items():
    #print(f"Bigram '{bigram}':")
    #for next_char, count in next_chars.items():
        #print(f"  '{next_char}': {count}")
    #print()  # Blank line for readability between bigrams

I then needed to whether I could generate text. I started by adding the `start_seed` which is `TH` to the generated text to ensure the generated text always begins with `TH`.  Next, I used this generated text as the bigram to look up the possible next characters and their weights. I then randomly selected the next character using the weights as probabilities. This next character is appended to the generated text. Printing the results shows a trigram with the last letter randomly generated from the model based on weights. Once I confirmed I can generated the next character. I'll create a loop of this process to repeatly generated a new letter until 10,000 characters are generated.

In [22]:
#initialize the generated text with the start seed
generatedtext = startseed

#get the last two characters in the generated text
bigram = generatedtext[-2:]

#get possible next characters and their counts
next_chars = list(trigram_dict[bigram].keys())
weights = list(trigram_dict[bigram].values())
    
#randomly select the next character based on weights
next_char = random.choices(next_chars, weights=weights, k=1)[0]
    
#next character to the generated text
generatedtext += next_char


#print the generated text with one additional character
print(generatedtext)

THE


Here, I essentially do the same as in the last cell, but this time, I add it to a loop that generates to the specified lenght of 10,000 charecters. This means the model will keep reading the last two letters in generated text and generating a third letter based on probabilities from the weights, untill the loop is broken.

In [23]:
#loop to generate characters until the length is reached
for _ in range(length):
    
    #get the last two characters in the generated text to form a bigram
    bigram = generatedtext[-2:]
    #get possible character and weights from trigram_dict
    next_chars = list(trigram_dict[bigram].keys()) #list of possible next characters for the bigram
    weights = list(trigram_dict[bigram].values())  #counts used as weights
    #select the next character based on the weights as probabilities
    next_char = random.choices(next_chars, weights=weights, k=1)[0]
    
    #chosen character to the generated text
    generatedtext += next_char

#print(generatedtext)


## Task 3: Analyze your model

First, I load the words from the text file into a data structure so I can use them for "comparison with the words generated by the model.

In [24]:
#set to store the comparison words
ComparisonSet = set()

#read the comparison file
with open(Comparisons, 'r') as f:
    #loop through the file and add each line to the set
    for line in f:
        ComparisonSet.add(line.strip())

#print(ComparisonSet)

I then use the `.split` method to split the generated text into words. I then initialize a variable to count the valid words. I  loop over the words in `generatedtext` and compare them to the words in the comparison set. If the word is vaild, I increment the vaild word counter. This is how I count the words in the generated text.

In [25]:
#split the generated text into words
generatedWords = generatedtext.split()

#count for vaild words
validWords = 0
for word in generatedWords:
    #check if the word is in the comparison set
    if word in ComparisonSet:
        #increment the valid word count
        validWords += 1

print(f"Valid words: {validWords}")

Valid words: 654


Now that I have the total amount of valid words, I can use simple average equation: `Average = Sum of Values/Number of Values`. First, I need to get the total words from the generated text, which I can do by getting its length. Then, I simply enter the valid numbers and total numbers variables into the equation to calculate the average.

In [26]:
#get total words in the generated text
totalWords = len(generatedWords)
#calculate the percentage of valid words
percentage = validWords / totalWords * 100

print(f"Total words: {totalWords}")
print(f"Valid words: {validWords}")
print(f"Percentage of valid words: {percentage:.2f}%")

Total words: 1822
Valid words: 654
Percentage of valid words: 35.89%


## Task 4:  Export your model as JSON

To export my model as JSON, I first open the a `.json` file with the `open` funtion. Once the file open, I can use the `json.dump` method of the `json` module to write the model to the file to the file with proper formatting (pretty print and indentation).

In [27]:
#writing the trigram dictionary to a json file
with open(jsonFile, 'w') as f:
    json.dump(trigram_dict, f, indent=4) #pretty print the json file

Once the model is saved to the `.json` file we need to verify that the model in the file is correct. I do this by loading the model from the file into a dictionary using `json.load`. I can then compare the loaded model with the original model and print r
the result to check if they match.

In [28]:
#verify the json file is the same as the dictionary
with open(jsonFile, 'r') as f:
    trigram_dict2 = json.load(f)

print(trigram_dict == trigram_dict2) #should print True

True


## Conclusion

This notebook has successfully walked through the steps of generating text using a trigram model, validating the generated words, and exporting the model into a JSON file. By following these steps outlined for these tasks I have created a model capable of generating text and evaluating its quality based on the real words provided.